In [647]:
%reset -f

In [648]:
#!/usr/bin/env python3
import psycopg2
import csv
import glob
import pandas as pd
from psycopg2 import sql
import time
from typing import Union
from datetime import datetime, timezone
import re
import numpy as np
import sys
import math
from tqdm import tqdm
import torch
pd.set_option("display.max_columns", None)

In [649]:
filename = 'Results_1673042557656.txt'

line_number = 1
simulation_parameter_counter = 1
a_spectrum_counter = 1
b_spectrum_counter = 1

lambdas = []
trans = []

thetas = []
pitchs = []
duty_cycles = []
fill_factors =[]
simulations = []
simulation = 1

num_lines = sum(1 for line in open(filename,'r'))
with open(filename, 'r') as file:
    for line in tqdm(file, total=num_lines):
        check1 = line_number == 1 + 4*(simulation_parameter_counter - 1)
        check2 = line_number == 2 + 4*(a_spectrum_counter - 1)
        # check3 = line_number == 3 + 4*(b_spectrum_counter - 1)
        if check1 or check2:
            # Get the grating coupler parameters
            if line_number == 1 + 4*(simulation_parameter_counter - 1):
                split_line = line.split(",")

                theta = float(split_line[0])
                pitch = float(split_line[1])
                duty_cycle = float(split_line[2])
                fill_factor = float(split_line[3])
                
                simulation_parameter_counter += 1

            if line_number == 2 + 4*(a_spectrum_counter - 1):  

                lambda_vals = []
                trans_vals = []

                # Use regular expression to find all words within brackets
                bracketed_words = re.findall(r'\(([^)]+)\)', line)

                # Split the bracketed words into separate words
                vals = [word.split(",") for word in bracketed_words]

                # sorted_vals = sorted(vals, key=lambda x: x[0])

                for value in vals:
                    lambda_vals.append(float(value[0])*1000000)
                    trans_vals.append(float(value[1]))
                    lambda_vals.reverse()
                    trans_vals.reverse()

                a_spectrum_counter += 1

            line_number += 1

            if check2:
                simulations.append(simulation)
                thetas.append(theta)
                pitchs.append(pitch)
                duty_cycles.append(duty_cycle)
                fill_factors.append(fill_factor)
                lambdas.append(lambda_vals)
                trans.append(trans_vals)
                simulation += 1
        else:
            line_number += 1

100%|██████████| 14996/14996 [00:00<00:00, 44267.43it/s]


In [650]:
data = {'simulation':simulations, 'theta':thetas, 'pitch':pitchs, 'duty_cycle':duty_cycles, 'fill_factor':fill_factors, 'lambda':lambdas, 'trans':trans}
dataframe = pd.DataFrame(data=data)
dataframe

,simulation,theta,pitch,duty_cycle,fill_factor,lambda,trans
0,1,10.0000,5.000000e-07,0.400000,0.200000,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000311453, 0.000577727, 0.000898426, 0.0012..."
1,2,10.0000,5.000000e-07,0.400000,0.244444,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[7.77445e-05, 3.47955e-05, 7.99542e-05, 0.0002..."
2,3,10.0000,5.000000e-07,0.400000,0.288889,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000554413, 0.000377262, 0.000212681, 9.5178..."
3,4,10.0000,5.000000e-07,0.400000,0.333333,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000735817, 0.000657348, 0.000511898, 0.0003..."
4,5,10.0000,5.000000e-07,0.400000,0.377778,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000588678, 0.000701883, 0.000749778, 0.0007..."
...,...,...,...,...,...,...,...
3744,3745,13.3333,1.277780e-06,0.577778,0.377778,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000105938, 0.000109693, 0.000117598, 0.0001..."
3745,3746,13.3333,1.277780e-06,0.577778,0.422222,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[5.09078e-05, 4.9765e-05, 4.96777e-05, 5.03599..."
3746,3747,13.3333,1.277780e-06,0.577778,0.466667,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.00020992, 0.000218771, 0.000208879, 0.00018..."
3747,3748,13.3333,1.277780e-06,0.577778,0.511111,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[6.36457e-05, 5.75334e-05, 4.6093e-05, 3.37111..."


In [651]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

In [652]:
from scipy.signal import find_peaks

In [653]:
num_rows = dataframe.shape[0]
for i in tqdm(range(num_rows)):
    simulation = dataframe.iloc[i]['simulation']
    wavelengths = dataframe.iloc[i]['lambda']
    transmissions = dataframe.iloc[i]['trans']

    peaks = find_peaks(transmissions, distance=150)
    if len(peaks) == 1:
        print(len(peaks))

    # plt.plot(wavelengths, transmissions, '.-')
    # plt.xlabel('lambda', fontsize=16)
    # plt.ylabel('Trans', fontsize=16)

    # plt.show()

    # print(simulation)

    # plt.clf()

100%|██████████| 3749/3749 [00:00<00:00, 7488.47it/s]


In [654]:
# i = 250 is a good curve
simulations = []
thetas = []
pitchs = []
duty_cycles = []
fill_factors = []
lambdas = []
trans = []

count = 0
for i in tqdm(range(num_rows)):
    wavelengths = dataframe.iloc[i]['lambda']
    transmissions = dataframe.iloc[i]['trans']

    indices = find_peaks(transmissions, distance=150, prominence=0.15, width=0.2)[0]
    # indices = find_peaks(transmissions, distance=150, prominence=0.1, width=0.2)[0]

    # indices = find_peaks(transmissions)[0]
    peaks_wavs = [wavelengths[j] for j in indices]
    peaks_trans = [transmissions[j] for j in indices]

    if len(peaks_wavs) > 0:
        # take out peaks that are less 0.1
        # take out peaks that lie outside 1.35 and 1.65
        check_wavelength = (max(peaks_wavs)) < 1.35 or (max(peaks_wavs) > 1.65)
        if len(indices) == 1 and max(peaks_trans) > 0.1 and check_wavelength is False:
            count = count + 1
            # plt.plot(wavelengths, transmissions, '.-')
            # plt.plot(peaks_wavs, peaks_trans, 'x')
            # plt.xlabel('lambda', fontsize=16)
            # plt.ylabel('Trans', fontsize=16)
            # plt.show()
            # plt.clf()

            simulations.append(dataframe.iloc[i]['simulation'])
            thetas.append(dataframe.iloc[i]['theta'])
            pitchs.append(dataframe.iloc[i]['pitch'])
            duty_cycles.append(dataframe.iloc[i]['duty_cycle'])
            fill_factors.append(dataframe.iloc[i]['fill_factor'])
            lambdas.append(dataframe.iloc[i]['lambda'])
            trans.append(dataframe.iloc[i]['trans'])
print(count)

100%|██████████| 3749/3749 [00:00<00:00, 5607.48it/s]

727


In [655]:
data = {'simulation':simulations, 'theta':thetas, 'pitch':pitchs, 'duty_cycle':duty_cycles, 'fill_factor':fill_factors, 'lambda':lambdas, 'trans':trans}
dataframe = pd.DataFrame(data=data)
dataframe 

,simulation,theta,pitch,duty_cycle,fill_factor,lambda,trans
0,160,10.0000,6.111110e-07,0.622222,0.600000,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000465189, 0.000483701, 0.000606855, 0.0010..."
1,167,10.0000,6.111110e-07,0.666667,0.466667,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.00374832, 0.00505441, 0.00703098, 0.0095631..."
2,168,10.0000,6.111110e-07,0.666667,0.511111,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.00194377, 0.00236592, 0.0030274, 0.00412377..."
3,169,10.0000,6.111110e-07,0.666667,0.555556,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.00102987, 0.000198873, 9.50273e-05, 0.00029..."
4,170,10.0000,6.111110e-07,0.666667,0.600000,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.00882811, 0.00531178, 0.00227771, 0.0006294..."
...,...,...,...,...,...,...,...
722,3719,13.3333,1.277780e-06,0.444444,0.555556,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000135358, 0.000116031, 9.9699e-05, 9.41271..."
723,3720,13.3333,1.277780e-06,0.444444,0.600000,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.00015286, 0.000155809, 0.000151682, 0.00013..."
724,3729,13.3333,1.277780e-06,0.488889,0.555556,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000305041, 0.00033168, 0.000346478, 0.00034..."
725,3730,13.3333,1.277780e-06,0.488889,0.600000,"[1.3, 1.30308, 1.30618, 1.30929, 1.31241, 1.31...","[0.000229818, 0.000235766, 0.000244474, 0.0002..."
